<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/MEDICAL_deepseekr1_api_demo_june2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://platform.deepseek.com/usage

## Libraries Installation

In [ ]:
!pip install colab-env -q
import colab_env

In [ ]:
# Cell 1: Install Necessary Libraries
!pip install crewai -q
!pip install 'crewai[tools]' -q
!pip install openai -q

## API setup

In [4]:
from google.colab import userdata

import os
deepseek_api_key = userdata.get('DEEPSEEK_API_KEY')

* The deepseek-chat model points to DeepSeek-V3-0324. You can invoke it by specifying model='deepseek-chat'.

* The deepseek-reasoner model points to DeepSeek-R1-0528. You can invoke it by specifying model='deepseek-reasoner'

https://huggingface.co/deepseek-ai/DeepSeek-R1-0528?utm_campaign=The%20Batch&utm_medium=email&_hsenc=p2ANqtz--M3NQ_S2i2a4-YigM_g7yAl65J52ZDkfTOAs4rKcQS5b4sBcaw8xHQxWdvtC9-m5_DNtdk92DS5nC5r5inVdTsTJvNZQ&_hsmi=364910056&utm_content=364905475&utm_source=hs_email

## Deepseek API Scenenarios

In [5]:
# Please install OpenAI SDK first: `pip3 install openai`

from openai import OpenAI

client = OpenAI(api_key=deepseek_api_key, base_url="https://api.deepseek.com")

response = client.chat.completions.create(
    #model="deepseek-chat",
    model="deepseek-reasoner",
    messages=[
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": "Hello"},
    ],
    stream=False
)

print(response.choices[0].message.content)

Hello! 👋 How can I assist you today?


In [6]:
def deepseek_chat(prompt,model):
  response = client.chat.completions.create(
    #model="deepseek-chat",
    #model="deepseek-reasoner",
    model=model,
    messages=[
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": prompt},
    ],
    stream=False
  )

  print()
  print("-" * 80)
  print('Model: %s'%model)
  print("-" * 80)
  print('\n')

  print("-" * 80)
  print('Question: %s'%prompt)
  print("-" * 80)
  print('\n')

  print('Answer: ')
  print(response.choices[0].message.content)
  #return message.content[0].text
  return response.choices[0].message.content

In [7]:
prompt = "What is the capital of France?"
response = deepseek_chat(prompt,'deepseek-chat')
print(response)


--------------------------------------------------------------------------------
Model: deepseek-chat
--------------------------------------------------------------------------------


--------------------------------------------------------------------------------
Question: What is the capital of France?
--------------------------------------------------------------------------------


Answer: 
The capital of France is **Paris**. It is one of the most famous and visited cities in the world, known for landmarks like the Eiffel Tower, the Louvre Museum, and Notre-Dame Cathedral.  

Would you like information on anything specific about Paris? 😊
The capital of France is **Paris**. It is one of the most famous and visited cities in the world, known for landmarks like the Eiffel Tower, the Louvre Museum, and Notre-Dame Cathedral.  

Would you like information on anything specific about Paris? 😊


In [8]:
prompt= 'A 65-year-old male presents with sudden onset severe chest pain radiating to his left arm. \
He reports shortness of breath and nausea. \
He has a history of hypertension and takes medication for it. \
His blood pressure is 160/100 mmHg and his heart rate is 110 bpm. \
What is the most likely diagnosis and what are the immediate steps for management?'

response = deepseek_chat(prompt,"deepseek-reasoner")
print(response)


--------------------------------------------------------------------------------
Model: deepseek-reasoner
--------------------------------------------------------------------------------


--------------------------------------------------------------------------------
Question: A 65-year-old male presents with sudden onset severe chest pain radiating to his left arm. He reports shortness of breath and nausea. He has a history of hypertension and takes medication for it. His blood pressure is 160/100 mmHg and his heart rate is 110 bpm. What is the most likely diagnosis and what are the immediate steps for management?
--------------------------------------------------------------------------------


Answer: 
Based on the presentation, the **most likely diagnosis is an acute coronary syndrome (ACS), specifically a myocardial infarction (heart attack)**. Here's why:

1.  **Classic Symptoms:** Sudden severe chest pain radiating to the left arm is highly characteristic of cardiac ischemia.

## TTP AGENT MEDICAL

In [9]:
import os
from crewai import Agent, Task, Crew, Process
from crewai.tools import BaseTool
from openai import OpenAI
from pydantic import Field

# --- Cell 2: Set Up DeepSeek API Key ---
try:
    from google.colab import userdata
    deepseek_api_key = userdata.get('DEEPSEEK_API_KEY')
except ImportError:
    print("google.colab not found. Assuming DEEPSEEK_API_KEY is set as an environment variable.")
    deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')

if not deepseek_api_key:
    raise ValueError("DEEPSEEK_API_KEY not found. Please set it in Colab secrets or as an environment variable.")

# --- Cell 3: Define Custom Tools for Medical Interaction ---

class MedicalDatabaseSearchTool(BaseTool):
    name: str = "MedicalDatabaseSearchTool"
    description: str = "A tool for searching medical databases for patient history, treatment guidelines, and research papers."

    def _run(self, query: str) -> str:
        return f"Medical database search results for '{query}': Patient records indicate no prior allergies. Latest research suggests treatment X for this condition. (Simulated)"

class ClinicalGuidelinesTool(BaseTool):
    name: str = "ClinicalGuidelinesTool"
    description: str = "A tool for retrieving current clinical guidelines and best practices for specific conditions."

    def _run(self, condition: str) -> str:
        return f"Clinical guidelines for {condition}: Standard protocol recommends initial dietary management and exercise, followed by medication if necessary. (Simulated)"

class DeepSeekMedicalChatTool(BaseTool):
    # BaseTool (via pydantic) expects these to be found here and will initialize them.
    name: str = "DeepSeekMedicalChatTool"
    description: str = "A powerful AI model for generating responses based on medical prompts and complex reasoning."

    # These are additional fields specific to DeepSeekMedicalChatTool's functionality
    client: OpenAI = Field(default=None, exclude=True)
    model_name: str = Field(default='deepseek-reasoner') # Default to reasoner for complex tasks

    # Custom __init__ to handle api_key and model_name.
    # It passes **data to super().__init__ but *not* name or description explicitly.
    def __init__(self, api_key: str, model_name: str = 'deepseek-reasoner', **data):
        # Call super().__init__ only with **data.
        super().__init__(**data)

        # Initialize OpenAI client with the provided api_key and DeepSeek base_url.
        self.client = OpenAI(api_key=api_key, base_url="https://api.deepseek.com")
        self.model_name = model_name

    def _run(self, prompt: str) -> str:
        if not isinstance(prompt, str):
            if isinstance(prompt, dict) and 'description' in prompt:
                actual_prompt = prompt['description']
            else:
                raise ValueError(f"Expected prompt to be a string or a dictionary with 'description', but received {type(prompt)}")
        else:
            actual_prompt = prompt

        try:
            response = self.client.chat.completions.create(
                model=self.model_name,
                messages=[
                    {"role": "system", "content": "You are a helpful medical AI assistant."},
                    {"role": "user", "content": actual_prompt},
                ],
                stream=False
            )
            return response.choices[0].message.content
        except Exception as e:
            return f"Error interacting with DeepSeek API: {e}"


# Instantiate medical tools with the retrieved deepseek_api_key
medical_database_search_tool = MedicalDatabaseSearchTool()
clinical_guidelines_tool = ClinicalGuidelinesTool()

# Instantiate DeepSeekMedicalChatTool. No need for tool_name parameter in instantiation.
deepseek_reasoner_tool_medical = DeepSeekMedicalChatTool(
    api_key=deepseek_api_key,
    model_name='deepseek-reasoner',
)

deepseek_chat_tool_medical = DeepSeekMedicalChatTool(
    api_key=deepseek_api_key,
    model_name='deepseek-chat',
)

patient_care_planner = Agent(
    role='Expert Patient Care Pathway Creator',
    goal='Develop comprehensive and personalized patient care pathways based on patient data, diagnoses, and medical guidelines.',
    backstory="""You are a seasoned medical professional with a knack for crafting optimal patient journeys.
                 You utilize advanced reasoning capabilities to integrate diverse medical information and patient preferences.""",
    tools=[deepseek_reasoner_tool_medical, medical_database_search_tool, clinical_guidelines_tool],
    verbose=False
)

clinical_data_analyst = Agent(
    role='Clinical Data Expert and Diagnostic Assistant',
    goal='Analyze patient symptoms, lab results, and medical history to suggest relevant diagnoses and potential complications.',
    backstory="""You are meticulous in reviewing clinical data, identifying patterns, and drawing insightful conclusions.
                 You ensure all relevant patient information is considered for accurate assessment and diagnostic support.""",
    tools=[deepseek_reasoner_tool_medical, medical_database_search_tool],
    verbose=False
)

resource_allocator = Agent(
    role='Medical Resource and Logistics Advisor',
    goal='Manage the allocation of medical resources (e.g., appointments, specialists, equipment) and provide practical logistical advice for patient care.',
    backstory="""You are meticulous with scheduling and resource management, ensuring efficient and timely patient care within available resources.
                 Your reasoning skills are critical for optimal resource allocation and patient flow management.""",
    tools=[deepseek_reasoner_tool_medical],
    #verbose=True
    verbose=False
)

medical_literature_reviewer = Agent(
    role='Expert Medical Literature Reviewer',
    goal='Provide accurate and timely summaries of the latest medical research and evidence for specific conditions or treatments.',
    backstory="""You are a research specialist adept at synthesizing concise and critical information from scientific publications.
                 You leverage medical search tools for comprehensive evidence gathering.""",
    tools=[medical_database_search_tool, deepseek_reasoner_tool_medical],
    #verbose=True
    verbose=False
)

treatment_coordinator = Agent(
    role='Efficient Treatment Plan Coordinator',
    goal='Plan and optimize all aspects of treatment implementation, including scheduling therapies, procedures, and follow-ups.',
    backstory="""You are a logistics expert with deep knowledge of various treatment modalities and service providers.
                 You use medical information for real-time data and reasoning to ensure smooth coordination of care.""",
    tools=[medical_database_search_tool, deepseek_reasoner_tool_medical],
    #verbose=True
    verbose=False
)

patient_experience_navigator = Agent(
    role='Patient Experience Navigator',
    goal='Discover unique, authentic, and patient-centered approaches to improve patient experience and well-being.',
    backstory="""You are a compassionate guide, always unearthing the most genuine experiences and support for patients.
                 You rely on medical search and reasoning for up-to-date information and personalized recommendations for patient well-being.""",
    tools=[medical_database_search_tool, deepseek_reasoner_tool_medical],
    #verbose=True
    verbose=False
)

# --- Cell 5: Define Your Tasks (Refactored for Medical Domain) ---

plan_patient_care_task = Task(
    description=(
        """Generate a detailed patient care pathway for a patient with the following profile:\n"""
        """- Patient ID: P12345\n"""
        """- Diagnosed Condition: Type 2 Diabetes\n"""
        """- Current Symptoms: Elevated blood sugar, fatigue, frequent urination.\n"""
        """- Medical History: Hypertension, moderate obesity.\n"""
        """- Patient Preferences: Prefers dietary management over immediate medication, active lifestyle.\n"""
        """- Incorporate latest clinical guidelines for Type 2 Diabetes management.\n"""
        """- Provide specific recommendations for diet, exercise, medication (if necessary), and follow-up schedule.\n"""
        """The output should be a markdown formatted document outlining the complete care pathway."""
    ),
    agent=patient_care_planner,
    output_file='patient_care_pathway.md',
    expected_output="""A markdown formatted, comprehensive patient care pathway including diagnosis summary, treatment plan, and follow-up schedule."""
)

get_clinical_guidelines_task = Task(
    description="Retrieve the current and recommended clinical guidelines for Type 2 Diabetes.",
    agent=medical_literature_reviewer,
    expected_output="""A summary of the current and recommended clinical guidelines for Type 2 Diabetes management."""
)

plan_treatment_options_task = Task(
    description="Identify the best treatment options (medication, lifestyle changes, therapies) for a patient with Type 2 Diabetes, considering their preferences and medical history.",
    agent=treatment_coordinator,
    expected_output="""A list of recommended treatment methods for Type 2 Diabetes based on patient profile and latest guidelines."""
)

find_niche_patient_support_task = Task(
    description="Based on the patient's condition (Type 2 Diabetes) and preferences (dietary management, active lifestyle), suggest unique, interest-aligned patient support activities and resources.",
    agent=patient_experience_navigator,
    expected_output="""A list of unique, interest-aligned patient support activities and resources for Type 2 Diabetes management."""
)

crew = Crew(
    agents=[
        patient_care_planner,
        clinical_data_analyst,
        resource_allocator,
        medical_literature_reviewer,
        treatment_coordinator,
        patient_experience_navigator
    ],
    tasks=[
        plan_patient_care_task,
        get_clinical_guidelines_task,
        plan_treatment_options_task,
        find_niche_patient_support_task
    ],
    verbose=False,
    process=Process.sequential
)

print("Starting the Patient Care Pathway Planning Crew (VERSION 2.0 - enhanced with clinical data and research integration)...")
result = crew.kickoff()

print("\n\n########################")
print("## Patient Care Pathway ##")
print("########################\n")
print(result)

Starting the Patient Care Pathway Planning Crew (VERSION 2.0 - enhanced with clinical data and research integration)...


########################
## Patient Care Pathway ##
########################

Here are some unique, interest-aligned patient support activities and resources for managing Type 2 Diabetes focusing on dietary management and an active lifestyle:

1. **Cooking Classes**: Enroll in hands-on cooking classes specializing in diabetic-friendly meals. This teaches patients how to prepare healthy, low-carb dishes using fresh ingredients.

2. **Nutrition Workshops**: Attend workshops that focus on meal planning, grocery shopping for healthy foods, and understanding food labels, emphasizing low-glycemic and high-fiber foods.

3. **Fitness Groups**: Join local exercise groups tailored for individuals with diabetes. Activities could include walking clubs, yoga sessions, or dance classes that encourage an active lifestyle while providing social support.

4. **Virtual Health Challen